<a href="https://colab.research.google.com/github/Gongmengjie/QA_Bert/blob/main/QA_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

Downloading...
From: https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1
To: /content/hw7_data.zip
100% 7.71M/7.71M [00:00<00:00, 122MB/s]


In [ ]:
!unzip -o hw7_data.zip

Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          


In [ ]:
!nvidia-smi

Mon Dec 20 12:07:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers==4.5.0

     |████████████████████████████████| 2.1 MB 8.7 MB/s 
     |████████████████████████████████| 895 kB 49.5 MB/s 
     |████████████████████████████████| 3.3 MB 40.8 MB/s 


In [ ]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset
# 这里的AdamW其实是有限制的Adam 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [22]:
# TODO 尝试换预训练模型[strong]
model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-chinese a

数据查看

In [23]:
with open("./hw7_dev.json", 'r') as load_f:
  load_dict = json.load(load_f)

In [29]:
question0 = load_dict['questions'][0]
question0

{'answer_end': 316,
 'answer_start': 312,
 'answer_text': '杜拜知識村',
 'id': 0,
 'paragraph_id': 538,
 'question_text': '哪一個訓練中心的設備被使用來訓練網絡城與媒體城勞工的未來知識?'}

In [34]:
paragraph_id = question0['paragraph_id']
answer_start = question0['answer_start']
answer_end = question0['answer_end']
question_text = question0['question_text']
question_text

'哪一個訓練中心的設備被使用來訓練網絡城與媒體城勞工的未來知識?'

In [35]:
load_dict['paragraphs'][paragraph_id][answer_start:answer_end+1]

'杜拜知識村'

In [ ]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

In [ ]:
# add_special_tokens设置为 False 因为当标记化的问题和段落在 datset __getitem__ 中组合时会添加特殊标记
train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)
# 忽视警告消息，因为在传递给模型之前，将在 datset __getitem__ 中进一步处理标记化序列

Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
class QA_Dataset(Dataset):

    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 150  # window的大小
        # self.doc_stride = 150
        # 步长变小，准确率由0.63提高到0.78
        self.doc_stride = 35  # 多个窗口的步长,采取多个窗口为了防止验证和预测时遗漏正确答案 

        # [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)  # 有多少个问题，也就是样本的大小

    def __getitem__(self, idx):
        question = self.questions[idx]  # 获得第idx个样本
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####          [strong]
        # 如何防止模型学习它不应该学习的东西

        if self.split == "train":
            # 将答案在paragraph_text 中的开始/结束位置转换为tokenized_pa​​ragraph 中的开始/结束位置  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])
            # 通过切片包含答案的段落部分来获得单个窗口
            # mid = (answer_start_token + answer_end_token) // 2
            # paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            # paragraph_end = paragraph_start + self.max_paragraph_len
            start = np.random.randint(0, self.max_paragraph_len - (answer_end_token - answer_start_token))
            # 画图可证明[paragraph_start:paragraph_start+1]必然包含正确答案
            paragraph_start = max(0, min(answer_start_token - start, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # 切分问题/段落并添加特殊标记 (101: CLS, 102: SEP)
            # 输入sequene1
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
            # 输入sequene2 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            # 将 tokenized_pa​​ragraph 中答案的开始/结束位置转换为窗口中的开始/结束位置
            # answer_start_token = len(input_ids_question) + start 画个图就能理解（1）
            # satrt = answer_start_token - paragraph_start（2）
            # 所以由（1）（2）得 answer_start_token += len(input_ids_question) - paragraph_start
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            # 填充序列并获得模型的输入 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token
        # Validation/Testing  # 验证集和测试集的处理数据方式一致
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
          
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
               
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)
    def padding(self, input_ids_question, input_ids_paragraph):
          padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
          input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
          token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
          attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
          return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 32

train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)


In [ ]:
for i in train_loader:
  print(i)

In [64]:
def evaluate(data, output):
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    # 预测结果会出现未知字符和空字符
    # 对于未知字符，使用更大的词库解决（bert-large）
    # 对于空字符，主要是预测的开始索引大于结束位置索引，导致无法预测字符，说明当前窗口段落不包含答案，可以舍去
    for k in range(num_of_windows):
  
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        prob = start_prob + end_prob

        if (start_index) >= end_index:
          continue
        
        if prob > max_prob:
            max_prob = prob
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
    
    return answer.replace(' ','')

In [65]:
num_epoch = 3
validation = True
logging_step = 100
learning_rate = 2e-5 # 学习率由1e-4到2e-5提高了3个百分点
optimizer = AdamW(model.parameters(), lr=learning_rate)
model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	

        data = [i.to(device) for i in data]
        
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
    
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        
        
        output.loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        step += 1
        # 增加学习率递减，准确率由0.5到0.6
        optimizer.param_groups[0]["lr"] -= learning_rate / (842 * num_epoch)
        
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()


print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


  0%|          | 0/842 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 0.931, acc = 0.615
Epoch 1 | Step 200 | loss = 0.802, acc = 0.664
Epoch 1 | Step 300 | loss = 0.719, acc = 0.704
Epoch 1 | Step 400 | loss = 0.702, acc = 0.693
Epoch 1 | Step 500 | loss = 0.671, acc = 0.709
Epoch 1 | Step 600 | loss = 0.663, acc = 0.717
Epoch 1 | Step 700 | loss = 0.646, acc = 0.726
Epoch 1 | Step 800 | loss = 0.596, acc = 0.748
Evaluating Dev Set ...


  0%|          | 0/3524 [00:00<?, ?it/s]

Validation | Epoch 1 | acc = 0.757


...

Validation | Epoch 2 | acc = 0.769


  0%|          | 0/842 [00:00<?, ?it/s]

Epoch 3 | Step 100 | loss = 0.341, acc = 0.820
Epoch 3 | Step 200 | loss = 0.328, acc = 0.835
Epoch 3 | Step 300 | loss = 0.324, acc = 0.838
Epoch 3 | Step 400 | loss = 0.351, acc = 0.834
Epoch 3 | Step 500 | loss = 0.326, acc = 0.839
Epoch 3 | Step 600 | loss = 0.314, acc = 0.846
Epoch 3 | Step 700 | loss = 0.336, acc = 0.830
Epoch 3 | Step 800 | loss = 0.348, acc = 0.832
Evaluating Dev Set ...


  0%|          | 0/3524 [00:00<?, ?it/s]

Validation | Epoch 3 | acc = 0.777
Saving Model ...


In [66]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output))

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/3493 [00:00<?, ?it/s]

Completed! Result is in result.csv
